## Importando

In [3]:
# Conectando com seu Drive

from google.colab import drive
drive.mount("/content/drive/")
import os


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [76]:
# Ativando diretório

os.chdir("/content/drive/MyDrive/Colab Notebooks/Dados/Datasets")

In [78]:
# Verificando

os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/Dados/Datasets'

# Trabalhando com previsões

In [26]:
# Importando bibliotecas

import pandas as pd


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [31]:
# Importando os dados

# passo 1 - Pre-processamento dos dados

# Carregar conjuntos de dados necessários

clientes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dados/Datasets/clientes.csv', sep= '\t')
produtos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dados/Datasets/produtos_traduzidos.csv', sep= '\t')
vendas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dados/Datasets/vendas.csv', sep= '\t')
detalhes_vendas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dados/Datasets/detalhes_vendas.csv', sep= '\t')


In [47]:
produtos.head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chás,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Xarope de Anis,1,2,12 - 550 ml bottles,10.00
3,4,Tempero Cajun do Chef Anton,2,2,48 - 6 oz jars,22.00
4,5,Mistura para Gumbo do Chef Anton,2,2,36 boxes,21.35


In [43]:
vendas.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [50]:
clientes.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [46]:
# Mesclar conjuntos de dados para obter um DataFrame com informações sobre as compras dos clientes
# mesclar vendas com detalhes das venda com as colunas em comum entre elas: OrderID

compras = pd.merge(vendas, detalhes_vendas, on="OrderID")
compras.head()


,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity
0,10248,90,5,1996-07-04,3,1,11,12
1,10248,90,5,1996-07-04,3,2,42,10
2,10248,90,5,1996-07-04,3,3,72,5
3,10249,81,6,1996-07-05,1,4,14,9
4,10249,81,6,1996-07-05,1,5,51,40


In [37]:
vendas.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [42]:
detalhes_vendas.head()

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5
3,4,10249,14,9
4,5,10249,51,40


In [49]:
# Segundo passo: Vamos mesclar o novo dataframe criado: COMPRAS com a tabela de produtos,
# assim podemos identificar também os nomes dos produtos

compras = pd.merge(compras, produtos[['ProductID', 'ProductName']], on='ProductID')

compras.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName
0,10248,90,5,1996-07-04,3,1,11,12,Queijo Cabrales
1,10296,46,6,1996-09-03,1,130,11,12,Queijo Cabrales
2,10327,24,2,1996-10-11,1,211,11,50,Queijo Cabrales
3,10353,59,7,1996-11-13,3,281,11,12,Queijo Cabrales
4,10365,3,3,1996-11-27,2,314,11,24,Queijo Cabrales


In [ ]:
# E agora para finalizar, vamos mesclar com Clientes.

compras = pd.merge(compras, clientes[['CustomerID', 'CustomerName']], on='CustomerID')

# depois que mesclar todas as informações, agora vamos deixar ele compacto apenas com os dados que importam

compras = compras[['CustomerName', 'ProductID', 'ProductName', 'Quantity']]




In [53]:
compras.head()

,CustomerName,ProductID,ProductName,Quantity
0,Wilman Kala,11,Queijo Cabrales,12
1,Wilman Kala,42,Mee Frito Hokkien de Singapura,10
2,Wilman Kala,72,Mozzarella di Giovanni,5
3,LILA-Supermercado,11,Queijo Cabrales,12
4,LILA-Supermercado,72,Mozzarella di Giovanni,3


In [57]:
# Passo 2: Matriz de Utilidade
# Agora vamos transpor, pivotar esse dataframe para transformar as colunas em linhas

# Criar uma matriz de utilidade com CustomerID como linhas (index),
# ProductName como colunas e Quantity como valores

matrix_utilidade = compras.pivot_table(values='Quantity', index='CustomerName', columns='ProductName', fill_value=0)

# Exibir a matriz de utilidade
matrix_utilidade.head()

ProductName,Arenque Defumado,Arenque Matjes Nord-Ost,Arenque Salgado,Arenque em Conserva,Biscoitos Longos Escoceses,Biscoitos Zaanse,Biscoitos de Chocolate Teatime,Bons Bolinhos de Pão Wimmers,Café Ipoh,Camembert Pierrot,...,Stout Steeleye,Tempero Cajun do Chef Anton,Tigres de Carnarvon,Tofu,Tofu Longlife,Torta Tourtière,Torta de Açúcar,Tunnbröd,Xarope de Anis,Xarope de Bordo
CustomerName,,,,,,,,,,,,,,,,,,,,,
Ana Trujillo Emparedados y helados,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0.0,0.0,0.0,0,0,0
Antonio Moreno Taquería,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0.0,0.0,0.0,0,0,0
Around the Horn,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0.0,0.0,0.0,0,0,0
B's Beverages,0,0,0,0,0,0,0,9,0,0.0,...,0,0,0,0,0.0,0.0,0.0,0,30,0
Berglunds snabbköp,0,0,0,0,0,0,0,0,0,15.0,...,0,0,0,0,0.0,0.0,0.0,0,0,0


In [58]:
# Passo 3: Similaridade entre clientes

# Calcular a similiaridade de consseno entre os clientes!

matriz_similaridade = cosine_similarity(matrix_utilidade)
matriz_similaridade

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.73165291,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.06059354,
        0.        ],
       [0.        , 0.73165291, 0.        , ..., 0.06059354, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [60]:
# Criando um dataframe com os nomes dos clientes e a matriz de similaridad

df_similaridade = pd.DataFrame(matriz_similaridade,
                               index=matrix_utilidade.index,
                               columns=matrix_utilidade.index)

df_similaridade

CustomerName,Ana Trujillo Emparedados y helados,Antonio Moreno Taquería,Around the Horn,B's Beverages,Berglunds snabbköp,Blondel père et fils,Bon app',Bottom-Dollar Marketse,Bólido Comidas preparadas,Centro comercial Moctezuma,...,Tortuga Restaurante,Tradição Hipermercados,Vaffeljernet,Victuailles en stock,Vins et alcools Chevalier,Wartian Herkku,Wellington Importadora,White Clover Markets,Wilman Kala,Wolski
CustomerName,,,,,,,,,,,,,,,,,,,,,
Ana Trujillo Emparedados y helados,1.000000,0.000000,0.000000,0.0,0.000000,0.174472,0.00000,0.263505,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.223043,0.000000,0.000000,0.0
Antonio Moreno Taquería,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.731653,0.0
Around the Horn,0.000000,0.000000,1.000000,0.0,0.104495,0.149215,0.00000,0.000000,0.145284,0.000000,...,0.178488,0.000000,0.000000,0.273724,0.077781,0.162233,0.023844,0.000000,0.000000,0.0
B's Beverages,0.000000,0.000000,0.000000,1.0,0.000000,0.076691,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Berglunds snabbköp,0.000000,0.000000,0.104495,0.0,1.000000,0.106417,0.00000,0.167790,0.431722,0.000000,...,0.154010,0.203947,0.000000,0.000000,0.000000,0.000000,0.236184,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wartian Herkku,0.000000,0.000000,0.162233,0.0,0.000000,0.000000,0.21013,0.000000,0.085794,0.153992,...,0.020495,0.045799,0.185422,0.161642,0.502378,1.000000,0.082504,0.000000,0.000000,0.0
Wellington Importadora,0.223043,0.000000,0.023844,0.0,0.236184,0.040471,0.00000,0.061124,0.000000,0.000000,...,0.010758,0.000000,0.040879,0.000000,0.000000,0.082504,1.000000,0.000000,0.000000,0.0
White Clover Markets,0.000000,0.000000,0.000000,0.0,0.000000,0.035365,0.00000,0.261718,0.144620,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.064521,0.000000,0.000000,1.000000,0.060594,0.0


In [61]:
# Passo 4: Gerar Recomendações

# Número de clientes semelhantes a considerar para recomendações
k = 3

# Número de recomendações!
numero_recomendacoes = 2

# Dicionário para armazenar recomendações para cada cliente
recomendacoes = {}

In [62]:
# Iterar por cada cliente na matriz de similaridade
for cliente in df_similaridade.index:
    # Encontrar os k clientes mais semelhantes ao cliente atual
    clientes_semelhantes = df_similaridade.loc[cliente].nlargest(k+1).iloc[1:].index

    # Encontrar produtos comprados pelos clientes semelhantes, mas não pelo cliente atual (assim não faria sentido, né, padawan :D)
    comprados_por_semelhantes = matrix_utilidade.loc[clientes_semelhantes].sum(axis=0)
    nao_comprados_pelo_atual = matrix_utilidade.loc[cliente] == 0
    produtos_recomendados = comprados_por_semelhantes[nao_comprados_pelo_atual].nlargest(numero_recomendacoes).index.tolist()

    # Armazenar as recomendações
    recomendacoes[cliente] = produtos_recomendados

In [64]:
recomendacoes

{'Ana Trujillo Emparedados y helados': ['Carne de Caranguejo de Boston',
  'Carneiro Alice'],
 'Antonio Moreno Taquería': ['Arenque Matjes Nord-Ost', 'Chang'],
 'Around the Horn': ['Chocolate Schoggi', 'Chartreuse Verde'],
 "B's Beverages": ['Gula Malacca', 'Pâté Chinois'],
 'Berglunds snabbköp': ['Xarope de Bordo', 'Gomas de Urso Gumbär'],
 'Blondel père et fils': ['Geitost', 'Maçãs Secas Manjimup'],
 "Bon app'": ['Fløtemysost', 'Pastel de Perth'],
 'Bottom-Dollar Marketse': ['Camembert Pierrot', 'Marmelada Sir Rodney'],
 'Bólido Comidas preparadas': ['Marmelada Sir Rodney', 'Nhoque da Vovó Alice'],
 'Centro comercial Moctezuma': ['Camembert Pierrot', 'Chucrute Rössle'],
 'Chop-suey Chinese': ['Gula Malacca', 'Camembert Pierrot'],
 'Comércio Mineiro': ['Chartreuse Verde', 'Nhoque da Vovó Alice'],
 'Consolidated Holdings': ['Camembert Pierrot', 'Chucrute Rössle'],
 'Die Wandernde Kuh': ['Raclette Courdavault', 'Fløtemysost'],
 'Drachenblut Delikatessend': ['Maxilaku', 'Tunnbröd'],
 'Du

In [65]:
# Converter o dicionário de recomendações para um DataFrame para melhor visualização

df_recomendacoes = pd.DataFrame(list(recomendacoes.items()), columns = ['CustomerName', 'Produtos recomendados'])

df_recomendacoes

,CustomerName,Produtos recomendados
0,Ana Trujillo Emparedados y helados,"[Carne de Caranguejo de Boston, Carneiro Alice]"
1,Antonio Moreno Taquería,"[Arenque Matjes Nord-Ost, Chang]"
2,Around the Horn,"[Chocolate Schoggi, Chartreuse Verde]"
3,B's Beverages,"[Gula Malacca, Pâté Chinois]"
4,Berglunds snabbköp,"[Xarope de Bordo, Gomas de Urso Gumbär]"
...,...,...
69,Wartian Herkku,"[Raclette Courdavault, Gorgonzola Telino]"
70,Wellington Importadora,"[Cerveja Klosterbier Rhönbräu, Arenque Salgado]"
71,White Clover Markets,"[Chartreuse Verde, Stout Steeleye]"
72,Wilman Kala,"[Queijo Gudbrandsdalsost, Fløtemysost]"


In [66]:
# Vamos tentar para os clientes do Brasil?
clientes.loc[clientes['Country'] == 'Brazil', 'CustomerName']

14          Comércio Mineiro
20        Familia Arquibaldo
30       Gourmet Lanchonetes
33             Hanari Carnes
60               Que Delícia
61             Queen Cozinha
66        Ricardo Adocicados
80    Tradição Hipermercados
87    Wellington Importadora
Name: CustomerName, dtype: object

In [67]:
df_recomendacoes.loc[df_recomendacoes['CustomerName'] == 'Comércio Mineiro'].values

array([['Comércio Mineiro',
        list(['Chartreuse Verde', 'Nhoque da Vovó Alice'])]], dtype=object)

In [69]:
df_recomendacoes.loc[df_recomendacoes['CustomerName'] == 'Tradição Hipermercados'].values

array([['Tradição Hipermercados',
        list(['Escargots de Bourgogne', 'Gorgonzola Telino'])]],
      dtype=object)

## Colocando em produção numa aplicação web

In [70]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.6 MB/s eta 0:

In [71]:
# Para funcionar no gradio, temos que criar uma função!
# Ela recebe o cliente e retorna as recomendações
def recomendar_produtos(cliente, k=3, numero_recomendacoes=2):
    """
    Função para gerar recomendações de produtos para um cliente específico.

    Parâmetros
    cliente (str): Nome do cliente para o qual as recomendações serão geradas.
    k (int): Número de clientes semelhantes a considerar para recomendações.
    numero_recomendacoes (int): Número de recomendações a serem geradas.

    Retorna:
    list: Lista de produtos recomendados para o cliente.
    """
    # Encontrar os k clientes mais semelhantes ao cliente atual
    clientes_semelhantes = df_similaridade.loc[cliente].nlargest(k+1).iloc[1:].index

    # Encontrar produtos comprados pelos clientes semelhantes, mas não pelo cliente atual
    comprados_por_semelhantes = matrix_utilidade.loc[clientes_semelhantes].sum(axis=0)
    nao_comprados_pelo_atual = matrix_utilidade.loc[cliente] == 0
    produtos_recomendados = comprados_por_semelhantes[nao_comprados_pelo_atual].nlargest(numero_recomendacoes).index.tolist()

    return produtos_recomendados


In [73]:
# Importar a biblioteca Gradio

import gradio as gr

# Criar a interface Gradio
def gui_recomendar_produtos(cliente):
    recomendacoes = recomendar_produtos(cliente)
    return ", ".join(recomendacoes)

# Lista de nomes de clientes
clientes_nomes = matrix_utilidade.index.tolist()

# Definir a interface
iface = gr.Interface(
    fn=gui_recomendar_produtos,
    inputs=gr.Dropdown(choices=clientes_nomes, label="Selecione um Cliente"),
    outputs=gr.Textbox(label="Produtos Recomendados"),
    live=False
)

# Iniciar an interface
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://66613be7483a068dc2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
